In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
from datetime import datetime
from matplotlib.dates import DateFormatter
import matplotlib.ticker as ticker

# Configuración de estilo para las gráficas
plt.style.use('ggplot')
sns.set(font_scale=1.2)
sns.set_style("whitegrid")

# Definir las rutas de los archivos CSV (deberás ajustar estas rutas a tu estructura de archivos)
# Esta es una estructura de ejemplo, ajusta según tu organización de archivos
data_files = {
    'JADE': {
        'Small': 'path/to/JADE_Small.csv',  # PID: 40492
        'Medium': 'path/to/JADE_Medium.csv',  # PID: 40428
        'Full': 'path/to/JADE_Full.csv'      # PID: 15476
    },
    'SPADE': {
        'Small': 'path/to/SPADE_Small.csv',  # PID: 49140
        'Medium': 'path/to/SPADE_Medium.csv', # PID: 46696
        'Full': 'path/to/SPADE_Full.csv'     # PID: 6284
    }
}

# PIDs para cada plataforma y escenario
pids = {
    'JADE': {
        'Small': 40492,
        'Medium': 40428,
        'Full': 15476
    },
    'SPADE': {
        'Small': 49140,
        'Medium': 46696,
        'Full': 6284
    }
}

# Función para cargar y preprocesar datos de Perfmon
def load_perfmon_data(filepath):
    """Carga y preprocesa un archivo CSV de Perfmon"""
    print(f"Cargando datos desde {filepath}...")
    
    try:
        # Leer el CSV
        df = pd.read_csv(filepath)
        
        # Renombrar la primera columna a 'Timestamp'
        df = df.rename(columns={df.columns[0]: 'Timestamp'})
        
        # Intentar extraer timestamps de la primera columna
        try:
            # Buscar patrones como MM/DD/YYYY HH:MM:SS.mmm en los datos
            date_pattern = r'(\d{1,4}[-/]\d{1,2}[-/]\d{1,4}\s\d{1,2}:\d{2}:\d{2}(?:\.\d+)?)'
            
            # Extraer timestamps si coinciden con el patrón
            timestamps = []
            for time_str in df['Timestamp']:
                match = re.search(date_pattern, str(time_str))
                if match:
                    try:
                        timestamp = pd.to_datetime(match.group(1))
                        timestamps.append(timestamp)
                    except:
                        timestamps.append(pd.NaT)
                else:
                    timestamps.append(pd.NaT)
                    
            # Si tenemos timestamps válidos, usarlos
            if not all(pd.isna(timestamps)):
                df['DateTimeIndex'] = timestamps
                df = df[~pd.isna(df['DateTimeIndex'])]
                print(f"Se extrajeron timestamps con éxito.")
            else:
                print("No se pudieron extraer timestamps de la primera columna. Usando secuencia numérica.")
                df['DateTimeIndex'] = pd.date_range(start=datetime.now(), periods=len(df), freq='S')
        except Exception as e:
            print(f"Error al procesar timestamps: {e}")
            print("Usando secuencia numérica con timestamps artificiales.")
            df['DateTimeIndex'] = pd.date_range(start=datetime.now(), periods=len(df), freq='S')
        
        # Calcular tiempo transcurrido en segundos desde el primer timestamp
        if 'DateTimeIndex' in df.columns:
            start_time = df['DateTimeIndex'].min()
            df['ElapsedTime'] = (df['DateTimeIndex'] - start_time).dt.total_seconds()
            print("Agregada columna 'ElapsedTime' en segundos.")
        
        return df
    
    except Exception as e:
        print(f"Error al cargar el archivo {filepath}: {e}")
        return None

# Función para encontrar columnas por PID o nombre de proceso
def find_columns_by_pid(df, pid, metric=None):
    """Encuentra columnas relacionadas con un PID específico y opcionalmente una métrica específica"""
    pid_cols = []
    
    # Buscar columnas que contengan el PID
    pid_str = str(pid)
    for col in df.columns:
        if pid_str in col:
            if metric is None or metric in col:
                pid_cols.append(col)
    
    # Si no se encontraron columnas con el PID, buscar por nombre de proceso (java para JADE, python para SPADE)
    if not pid_cols:
        # Para JADE (Java)
        java_cols = []
        for col in df.columns:
            if 'java' in col.lower():
                if metric is None or metric in col:
                    java_cols.append(col)
        
        # Para SPADE (Python)
        python_cols = []
        for col in df.columns:
            if 'python' in col.lower():
                if metric is None or metric in col:
                    python_cols.append(col)
        
        # Determinar qué conjunto usar basado en el PID (asumiendo que PIDs de JADE y SPADE corresponden a procesos específicos)
        is_jade_pid = any(pid == p for p in pids['JADE'].values())
        if is_jade_pid and java_cols:
            pid_cols = java_cols
        elif not is_jade_pid and python_cols:
            pid_cols = python_cols
    
    return pid_cols

# Función para limpiar nombres de columna para visualización
def clean_column_name(col_name, platform, scenario):
    """Extrae el nombre de proceso y métrica de un nombre de columna para visualización más limpia"""
    try:
        # Patrón para encontrar proceso(nombre)\métrica o formatos similares
        match = re.search(r'Process\((.*?)\)\\(.*)', col_name)
        if match:
            process_name = match.group(1)
            metric = match.group(2)
            return f"{platform} ({scenario}) - {process_name} - {metric}"
        else:
            parts = col_name.split('\\')
            if len(parts) > 1:
                return f"{platform} ({scenario}) - {parts[-1]}"
            else:
                return f"{platform} ({scenario}) - {col_name}"
    except:
        return f"{platform} ({scenario}) - {col_name}"

# Función para crear gráficas comparativas de CPU
def plot_cpu_comparison(dataframes, platform_colors=None):
    """
    Crea una gráfica comparativa de uso de CPU entre JADE y SPADE para los tres escenarios
    
    Args:
        dataframes: diccionario de DataFrames organizados por plataforma y escenario
        platform_colors: diccionario opcional de colores para cada plataforma
    """
    if platform_colors is None:
        platform_colors = {'JADE': 'blue', 'SPADE': 'green'}
    
    plt.figure(figsize=(16, 8))
    
    # Estilos de línea para diferentes escenarios
    styles = {'Small': '-', 'Medium': '--', 'Full': '-.'}
    
    for platform in dataframes:
        for scenario in dataframes[platform]:
            df = dataframes[platform][scenario]
            if df is not None:
                pid = pids[platform][scenario]
                cpu_cols = find_columns_by_pid(df, pid, '% Processor Time')
                
                if cpu_cols:
                    for col in cpu_cols[:1]:  # Tomar solo la primera columna para no saturar la gráfica
                        label = f"{platform} - {scenario}"
                        plt.plot(df['ElapsedTime'], df[col], 
                                 label=label, 
                                 color=platform_colors[platform], 
                                 linestyle=styles[scenario],
                                 linewidth=2)
    
    plt.title('Comparación de Uso de CPU: JADE vs SPADE', fontsize=18)
    plt.xlabel('Tiempo Transcurrido (segundos)', fontsize=14)
    plt.ylabel('% Tiempo de Procesador', fontsize=14)
    plt.grid(True, alpha=0.3)
    plt.legend(fontsize=12)
    plt.tight_layout()
    
    # Guardar la gráfica
    plt.savefig('jade_vs_spade_cpu_comparison.png', dpi=300)
    plt.show()

# Función para crear gráficas comparativas de memoria
def plot_memory_comparison(dataframes, memory_metric='Working Set', platform_colors=None):
    """
    Crea una gráfica comparativa de uso de memoria entre JADE y SPADE para los tres escenarios
    
    Args:
        dataframes: diccionario de DataFrames organizados por plataforma y escenario
        memory_metric: métrica de memoria a comparar ('Working Set', 'Private Bytes', etc.)
        platform_colors: diccionario opcional de colores para cada plataforma
    """
    if platform_colors is None:
        platform_colors = {'JADE': 'blue', 'SPADE': 'green'}
    
    plt.figure(figsize=(16, 8))
    
    # Estilos de línea para diferentes escenarios
    styles = {'Small': '-', 'Medium': '--', 'Full': '-.'}
    
    for platform in dataframes:
        for scenario in dataframes[platform]:
            df = dataframes[platform][scenario]
            if df is not None:
                pid = pids[platform][scenario]
                memory_cols = find_columns_by_pid(df, pid, memory_metric)
                
                if memory_cols:
                    for col in memory_cols[:1]:  # Tomar solo la primera columna para no saturar la gráfica
                        # Convertir bytes a MB para mejor legibilidad
                        label = f"{platform} - {scenario}"
                        plt.plot(df['ElapsedTime'], df[col]/1024/1024, 
                                 label=label, 
                                 color=platform_colors[platform], 
                                 linestyle=styles[scenario],
                                 linewidth=2)
    
    plt.title(f'Comparación de Uso de Memoria ({memory_metric}): JADE vs SPADE', fontsize=18)
    plt.xlabel('Tiempo Transcurrido (segundos)', fontsize=14)
    plt.ylabel('Memoria (MB)', fontsize=14)
    plt.grid(True, alpha=0.3)
    plt.legend(fontsize=12)
    plt.tight_layout()
    
    # Guardar la gráfica
    plt.savefig(f'jade_vs_spade_{memory_metric.lower().replace(" ", "_")}_comparison.png', dpi=300)
    plt.show()

# Función para calcular métricas de rendimiento
def calculate_performance_metrics(dataframes):
    """
    Calcula métricas de rendimiento para cada plataforma y escenario
    
    Args:
        dataframes: diccionario de DataFrames organizados por plataforma y escenario
    
    Returns:
        Un DataFrame con métricas de rendimiento
    """
    metrics = []
    
    for platform in dataframes:
        for scenario in dataframes[platform]:
            df = dataframes[platform][scenario]
            if df is not None:
                pid = pids[platform][scenario]
                
                # Obtener métricas de CPU
                cpu_cols = find_columns_by_pid(df, pid, '% Processor Time')
                if cpu_cols:
                    cpu_col = cpu_cols[0]  # Usar la primera columna de CPU
                    cpu_avg = df[cpu_col].mean()
                    cpu_max = df[cpu_col].max()
                    cpu_std = df[cpu_col].std()
                else:
                    cpu_avg = cpu_max = cpu_std = float('nan')
                
                # Obtener métricas de memoria privada
                mem_private_cols = find_columns_by_pid(df, pid, 'Private Bytes')
                if mem_private_cols:
                    mem_private_col = mem_private_cols[0]
                    mem_private_avg = df[mem_private_col].mean() / (1024 * 1024)  # En MB
                    mem_private_max = df[mem_private_col].max() / (1024 * 1024)  # En MB
                else:
                    mem_private_avg = mem_private_max = float('nan')
                
                # Obtener métricas de memoria de trabajo
                mem_ws_cols = find_columns_by_pid(df, pid, 'Working Set')
                if mem_ws_cols:
                    mem_ws_col = mem_ws_cols[0]
                    mem_ws_avg = df[mem_ws_col].mean() / (1024 * 1024)  # En MB
                    mem_ws_max = df[mem_ws_col].max() / (1024 * 1024)  # En MB
                else:
                    mem_ws_avg = mem_ws_max = float('nan')
                
                # Calcular duración total
                duration = df['ElapsedTime'].max() - df['ElapsedTime'].min()
                
                # Agregar a la lista de métricas
                metrics.append({
                    'Platform': platform,
                    'Scenario': scenario,
                    'CPU_Avg (%)': cpu_avg,
                    'CPU_Max (%)': cpu_max,
                    'CPU_StdDev': cpu_std,
                    'Mem_Private_Avg (MB)': mem_private_avg,
                    'Mem_Private_Max (MB)': mem_private_max,
                    'Mem_WorkingSet_Avg (MB)': mem_ws_avg,
                    'Mem_WorkingSet_Max (MB)': mem_ws_max,
                    'Duration (sec)': duration
                })
    
    return pd.DataFrame(metrics)

# Función para crear un panel de control comparativo
def create_comparison_dashboard(dataframes, platform_colors=None):
    """
    Crea un panel de control con múltiples gráficas comparando JADE y SPADE
    
    Args:
        dataframes: diccionario de DataFrames organizados por plataforma y escenario
        platform_colors: diccionario opcional de colores para cada plataforma
    """
    if platform_colors is None:
        platform_colors = {'JADE': 'blue', 'SPADE': 'green'}
    
    # Configurar el panel de control con 2x2 gráficas
    fig, axes = plt.subplots(2, 2, figsize=(18, 14))
    
    # Estilos de línea para diferentes escenarios
    styles = {'Small': '-', 'Medium': '--', 'Full': '-.'}
    
    # 1. Gráfica de CPU - Escenario Small
    ax = axes[0, 0]
    for platform in dataframes:
        df = dataframes[platform]['Small']
        if df is not None:
            pid = pids[platform]['Small']
            cpu_cols = find_columns_by_pid(df, pid, '% Processor Time')
            
            if cpu_cols:
                for col in cpu_cols[:1]:
                    ax.plot(df['ElapsedTime'], df[col], 
                             label=f"{platform} - Small", 
                             color=platform_colors[platform],
                             linewidth=2)
    
    ax.set_title('Uso de CPU - Escenario Small', fontsize=14)
    ax.set_ylabel('% Tiempo de Procesador', fontsize=12)
    ax.set_xlabel('Tiempo Transcurrido (segundos)', fontsize=12)
    ax.grid(True, alpha=0.3)
    ax.legend()
    
    # 2. Gráfica de CPU - Escenario Medium
    ax = axes[0, 1]
    for platform in dataframes:
        df = dataframes[platform]['Medium']
        if df is not None:
            pid = pids[platform]['Medium']
            cpu_cols = find_columns_by_pid(df, pid, '% Processor Time')
            
            if cpu_cols:
                for col in cpu_cols[:1]:
                    ax.plot(df['ElapsedTime'], df[col], 
                             label=f"{platform} - Medium", 
                             color=platform_colors[platform],
                             linewidth=2)
    
    ax.set_title('Uso de CPU - Escenario Medium', fontsize=14)
    ax.set_ylabel('% Tiempo de Procesador', fontsize=12)
    ax.set_xlabel('Tiempo Transcurrido (segundos)', fontsize=12)
    ax.grid(True, alpha=0.3)
    ax.legend()
    
    # 3. Gráfica de Memoria - Escenario Medium
    ax = axes[1, 0]
    for platform in dataframes:
        df = dataframes[platform]['Medium']
        if df is not None:
            pid = pids[platform]['Medium']
            mem_cols = find_columns_by_pid(df, pid, 'Working Set')
            
            if mem_cols:
                for col in mem_cols[:1]:
                    ax.plot(df['ElapsedTime'], df[col]/1024/1024, 
                             label=f"{platform} - Medium", 
                             color=platform_colors[platform],
                             linewidth=2)
    
    ax.set_title('Uso de Memoria (Working Set) - Escenario Medium', fontsize=14)
    ax.set_ylabel('Memoria (MB)', fontsize=12)
    ax.set_xlabel('Tiempo Transcurrido (segundos)', fontsize=12)
    ax.grid(True, alpha=0.3)
    ax.legend()
    
    # 4. Gráfica de CPU - Escenario Full
    ax = axes[1, 1]
    for platform in dataframes:
        df = dataframes[platform]['Full']
        if df is not None:
            pid = pids[platform]['Full']
            cpu_cols = find_columns_by_pid(df, pid, '% Processor Time')
            
            if cpu_cols:
                for col in cpu_cols[:1]:
                    ax.plot(df['ElapsedTime'], df[col], 
                             label=f"{platform} - Full", 
                             color=platform_colors[platform],
                             linewidth=2)
    
    ax.set_title('Uso de CPU - Escenario Full', fontsize=14)
    ax.set_ylabel('% Tiempo de Procesador', fontsize=12)
    ax.set_xlabel('Tiempo Transcurrido (segundos)', fontsize=12)
    ax.grid(True, alpha=0.3)
    ax.legend()
    
    plt.tight_layout()
    plt.savefig('jade_vs_spade_dashboard.png', dpi=300)
    plt.show()

# Función para realizar un análisis estadístico de los datos
def perform_statistical_analysis(metrics_df):
    """
    Realiza un análisis estadístico comparativo entre JADE y SPADE
    
    Args:
        metrics_df: DataFrame con métricas calculadas
    """
    # Análisis por plataforma
    platform_comparison = metrics_df.groupby('Platform').mean()
    platform_comparison = platform_comparison.drop('Duration (sec)', axis=1)  # Eliminar duración para evitar contaminar el análisis
    
    print("\n=== Comparación de Rendimiento Promedio por Plataforma ===")
    print(platform_comparison)
    
    # Análisis por escenario y plataforma
    scenario_platform_comparison = metrics_df.pivot_table(
        index='Scenario', 
        columns='Platform',
        values=['CPU_Avg (%)', 'CPU_Max (%)', 'Mem_WorkingSet_Avg (MB)', 'Duration (sec)']
    )
    
    print("\n=== Comparación de Rendimiento por Escenario y Plataforma ===")
    print(scenario_platform_comparison)
    
    # Calcular mejora relativa (porcentaje de diferencia)
    # Valor positivo => JADE es mejor, valor negativo => SPADE es mejor
    relative_improvement = pd.DataFrame()
    
    for scenario in metrics_df['Scenario'].unique():
        jade_data = metrics_df[(metrics_df['Platform'] == 'JADE') & (metrics_df['Scenario'] == scenario)]
        spade_data = metrics_df[(metrics_df['Platform'] == 'SPADE') & (metrics_df['Scenario'] == scenario)]
        
        if not jade_data.empty and not spade_data.empty:
            jade_row = jade_data.iloc[0]
            spade_row = spade_data.iloc[0]
            
            # Para CPU y memoria, menor es mejor
            cpu_diff = (spade_row['CPU_Avg (%)'] - jade_row['CPU_Avg (%)']) / spade_row['CPU_Avg (%)'] * 100
            mem_diff = (spade_row['Mem_WorkingSet_Avg (MB)'] - jade_row['Mem_WorkingSet_Avg (MB)']) / spade_row['Mem_WorkingSet_Avg (MB)'] * 100
            
            # Para duración, menor es mejor
            time_diff = (spade_row['Duration (sec)'] - jade_row['Duration (sec)']) / spade_row['Duration (sec)'] * 100
            
            relative_improvement = pd.concat([relative_improvement, pd.DataFrame({
                'Scenario': [scenario],
                'CPU Improvement (%)': [cpu_diff],
                'Memory Improvement (%)': [mem_diff],
                'Time Improvement (%)': [time_diff]
            })], ignore_index=True)
    
    print("\n=== Mejora Relativa de JADE comparado con SPADE (% de mejora) ===")
    print("Nota: Valores positivos indican que JADE es mejor, valores negativos que SPADE es mejor")
    print(relative_improvement)
    
    # Crear visualizaciones de la mejora relativa
    plt.figure(figsize=(12, 8))
    
    # Preparar datos para gráfica de barras
    scenarios = relative_improvement['Scenario']
    cpu_improvement = relative_improvement['CPU Improvement (%)']
    mem_improvement = relative_improvement['Memory Improvement (%)']
    time_improvement = relative_improvement['Time Improvement (%)']
    
    x = np.arange(len(scenarios))
    width = 0.25
    
    # Crear barras
    plt.bar(x - width, cpu_improvement, width, label='CPU', color='#3498db')
    plt.bar(x, mem_improvement, width, label='Memoria', color='#2ecc71')
    plt.bar(x + width, time_improvement, width, label='Tiempo', color='#e74c3c')
    
    # Añadir detalles a la gráfica
    plt.axhline(y=0, color='black', linestyle='-', alpha=0.3)
    plt.xlabel('Escenario', fontsize=14)
    plt.ylabel('Mejora (%) - JADE vs SPADE', fontsize=14)
    plt.title('Comparación de Rendimiento Relativo: JADE vs SPADE', fontsize=16)
    plt.xticks(x, scenarios)
    plt.legend()
    
    # Añadir etiquetas de valor a las barras
    for i, v in enumerate(cpu_improvement):
        plt.text(i - width, v + (5 if v >= 0 else -10), f"{v:.1f}%", ha='center', fontsize=9)
    for i, v in enumerate(mem_improvement):
        plt.text(i, v + (5 if v >= 0 else -10), f"{v:.1f}%", ha='center', fontsize=9)
    for i, v in enumerate(time_improvement):
        plt.text(i + width, v + (5 if v >= 0 else -10), f"{v:.1f}%", ha='center', fontsize=9)
    
    plt.tight_layout()
    plt.savefig('jade_vs_spade_relative_improvement.png', dpi=300)
    plt.show()

# Función principal para ejecutar el análisis completo
def main():
    print("=== Iniciando Análisis Comparativo de Rendimiento JADE vs SPADE ===")
    
    # Cargar los datos
    loaded_data = {}
    for platform in data_files:
        loaded_data[platform] = {}
        for scenario in data_files[platform]:
            file_path = data_files[platform][scenario]
            loaded_data[platform][scenario] = load_perfmon_data(file_path)
    
    # Verificar si se cargaron datos correctamente
    data_loaded = False
    for platform in loaded_data:
        for scenario in loaded_data[platform]:
            if loaded_data[platform][scenario] is not None:
                data_loaded = True
                break
        if data_loaded:
            break
    
    if not data_loaded:
        print("¡Error! No se pudo cargar ningún archivo de datos. Verifica las rutas y formatos.")
        return
    
    # Colores para cada plataforma
    platform_colors = {'JADE': '#1e88e5', 'SPADE': '#43a047'}
    
    # Generar visualizaciones comparativas
    print("\n=== Generando Visualizaciones Comparativas ===")
    plot_cpu_comparison(loaded_data, platform_colors)
    plot_memory_comparison(loaded_data, 'Working Set', platform_colors)
    plot_memory_comparison(loaded_data, 'Private Bytes', platform_colors)
    create_comparison_dashboard(loaded_data, platform_colors)
    
    # Calcular métricas de rendimiento
    print("\n=== Calculando Métricas de Rendimiento ===")
    metrics_df = calculate_performance_metrics(loaded_data)
    print("\nResumen de Métricas de Rendimiento:")
    print(metrics_df)
    
    # Guardar resultados en CSV
    metrics_df.to_csv('jade_vs_spade_performance_metrics.csv', index=False)
    print("Métricas guardadas en 'jade_vs_spade_performance_metrics.csv'")
    
    # Realizar análisis estadístico
    print("\n=== Realizando Análisis Estadístico ===")
    perform_statistical_analysis(metrics_df)
    
    print("\n=== Análisis Completado ===")
    print("Las visualizaciones se han guardado como archivos PNG en el directorio actual.")

if __name__ == "__main__":
    main()